In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path
import statsmodels.api as sm
import torch
from torch import nn
from tqdm.notebook import tqdm
from IPython.display import display, HTML
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX

current_dir = Path.cwd()
import sys
import gc

sys.path.append(str(current_dir.parent))
from utils import get_competition_data_path, submit
%matplotlib inline

In [2]:
sns.set()
plt.rcParams["figure.figsize"] = 20, 10

In [3]:
path_dict = get_competition_data_path("m5-forecasting-accuracy")
competition_path = path_dict.get("competition_path")
train_path = path_dict.get("train_path")
submission_path = path_dict.get("sample_submission_path")
calendar_path = competition_path / "calendar.csv"
sell_prices_path = competition_path / "sell_prices.csv"

In [4]:
def rmse(y_true, y_pred):
    
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [5]:
sales = pd.read_csv(train_path)

sales = sales.set_index('id').drop(columns=['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])
sales.index = sales.index.to_series().str.rsplit('_', 1, expand=True)[0].rename('id')
sales.head()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
id,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1,0,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
HOBBIES_1_002_CA_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
HOBBIES_1_003_CA_1,0,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
HOBBIES_1_004_CA_1,0,0,0,0,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
HOBBIES_1_005_CA_1,0,0,0,0,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


Remove days that should have no values (Xmas)

In [6]:
total_sales = sales.sum(axis=0)
outliers = total_sales.loc[total_sales < 10000].index.tolist()

In [7]:
outliers

['d_331', 'd_697', 'd_1062', 'd_1427', 'd_1792']

In [ ]:
sales = sales.loc[:, ~sales.columns.isin(outliers)]
total_sales = total_sales.loc[~total_sales.index.isin(outliers)]

In [ ]:
n_vals = 28

In [ ]:
y_train_df, y_val_df = sales.iloc[:, :-n_vals], sales.iloc[:, -n_vals:]
y_train, y_val = y_train_df.to_numpy(), y_val_df.to_numpy()

In [ ]:
total_y_train_df, total_y_val_df = total_sales.iloc[:-n_vals], total_sales.iloc[-n_vals:]
total_y_train, total_y_val = total_y_train_df.to_numpy(), total_y_val_df.to_numpy()

In [ ]:
n_trains = y_train.shape[1]
print(y_train.shape)
print(y_val.shape)

- Naive
- Seasonal Naive
    - 7 days
    - 28 days
    - 365 days
- Simple Exponential Smoothing
    - $Y_{t + 1} = \alpha * Y_t + (1 - \alpha) * Y_{t-1}$
    - $\alpha = [0.1,  0.3]$
- Moving Averages
    - 7 days
    - 28 days
    - 365 days
- ES (Top-down)
    - Last 28 days to estimate the proportion
- Arima (Top-down)
    - Last 28 days to estimate the proportion
- Average of ES and ARIMA

### Naive
$y_t = y_{t-1}$

In [ ]:
last_y = y_train[:, -1].reshape(-1, 1)
y_pred_naive = np.repeat(last_y, 28, axis=1)

In [ ]:
print('RMSE for Naive:', rmse(y_val, y_pred_naive))

### Seasonal Naive
- 7 days
- 28 days
- 365 days

In [ ]:
last_7_y = y_train[:, -7:]
y_pred_snaive_7 = np.tile(last_7_y, 4)

y_pred_snaive_28 = y_train[:, -28:]

y_pred_snaive_365 = y_train[:, -365: -365+28]

all_y_pred_snaive = [y_pred_snaive_7, y_pred_snaive_28, y_pred_snaive_365]

In [ ]:
print('RMSE for sNaive lag 7:', rmse(y_val, y_pred_snaive_7))
print('RMSE for sNaive lag 28:', rmse(y_val, y_pred_snaive_28))
print('RMSE for sNaive lag 365:', rmse(y_val, y_pred_snaive_365))
print('RMSE for average sNaive:', rmse(y_val, np.sum(all_y_pred_snaive, axis=0) / 3))

- Moving from Naive prediction to Seasonal Naive already improves the RMSE.
- Taking average improves significantly

### Simple Exponential Smoothing
- $Y_{(t + 1)|t} = \alpha * Y_t + (1 - \alpha) * Y_{t|t-1}$
- $\alpha = [0.1,  0.3]$

In [ ]:
all_y_pred_ses = []

# Loop through each alpha to generate forecast
for alpha in [0.1, 0.2, 0.3]:
    # Get empty array to store prediction value
    y_pred_ses = np.zeros_like(y_val)
    
    # Train array to extend period by period
    y_train_ses = y_train.copy()
    
    # Predict period by period
    for i in range(n_vals):
        # Get the smoothing coefficients
        power_arr = np.arange(0, n_trains + i)[::-1]
        coeff = (alpha * np.power(alpha, power_arr)).reshape(1, -1)
        
        # Get predicton
        this_ses = np.multiply(y_train_ses, coeff).sum(axis=1)
        
        # Add back to the array
        y_train_ses = np.concatenate([y_train_ses, this_ses.reshape(-1, 1)], axis=1)
        y_pred_ses[:, i] = this_ses
    
    print(f'RMSE for SES with alpha {alpha}:', rmse(y_val, y_pred_ses))
    
    all_y_pred_ses.append(y_pred_ses)
    
print(f'RMSE for average SES:', rmse(y_val, np.sum(all_y_pred_ses, axis=0) / 3))

- The SES doesn't work so well compared to other benchmarks
- Its usage is more appropriate for data with no clear trend and seasonality
- The saels data that we have clearly exhibits trend and seasonality

### Moving Averages
- 7 days
- 28 days
- 365 days

In [ ]:
all_y_pred_ma = []

# Loop through each alpha to generate forecast
for window in [7, 28, 365]:
    # Get empty array to store prediction value
    y_pred_ma = np.zeros_like(y_val)
    
    # Train array to extend period by period
    y_train_ma = y_train.copy()
    
    # Predict period by period
    for i in range(n_vals):
        # Get predicton
        this_ma = y_train_ma[:, -window:].mean(axis=1)
        
        # Add back to the array
        y_train_ma = np.concatenate([y_train_ma, this_ma.reshape(-1, 1)], axis=1)
        y_pred_ma[:, i] = this_ma
    
    print(f'RMSE for MA with window {window}:', rmse(y_val, y_pred_ma))
    
    all_y_pred_ma.append(y_pred_ma)
    
print(f'RMSE for average MA:', rmse(y_val, np.sum(all_y_pred_ma, axis=0) / 3))

- Moving Average is able to take in account the trend and performs pretty well

### ES and SARIMA (Top-down)
- Estimate the total sales with ES and ARIMA
- Trickle down using proportion of sales to get the lowest level
- Last 28 days to estimate the proportion
- Average of ES and ARIMA

In [ ]:
total_sales.plot()

In [ ]:
proportion = np.divide(y_train, total_y_train)[:, -28:].mean(axis=1).reshape(-1, 1)

#### ES

In [ ]:
all_y_pred_es = []
for seasonality in [7, 28, 365]:
    es = ExponentialSmoothing(total_y_train, trend='add', damped=True, seasonal='add', seasonal_periods=seasonality)
    es.fit()
    pred_es = es.predict(es.params, start=n_trains + 1, end=n_trains + 28)
    y_pred_es = np.multiply(proportion, pred_es)
    
    print(f'RMSE for top-down ES with seasonality cycle of {seasonality} days:', rmse(y_val, y_pred_es))
    
    all_y_pred_es.append(y_pred_es)
    
print(f'RMSE for average top-down ES:', rmse(y_val, np.sum(all_y_pred_es, axis=0) / 3))

#### SARIMA

In [ ]:
model = SARIMAX(
    total_y_train,
    order=(1, 1, 1),
    seasonal_order=(1, 1, 1, 7),
    enforce_stationarity=False,
    enforce_invertibility=False,
)
result = model.fit()
print(results.summary().tables[1])

In [ ]:
results.plot_diagnostics(figsize=(16, 8));

In [ ]:
pred_arima = results.predict(start=n_trains + 1, end=n_trains + 28)
y_pred_sarima = np.multiply(proportion, pred_arima)

print(f'RMSE for SARIMA:', rmse(y_val, y_pred_sarima))